# LiricAI

Aqui é mostrado um pequeno projecto em que se usa uma Recurrent Neural Network (segundo https://gilberttanner.com/blog/generating-text-using-a-recurrent-neuralnetwork) para criar a letra de uma música a partir de um determinado conjunto de letras já existente. 

## Os dados

Os ficheiros utilizados encontram-se disponíveis no repositório de Github como ficheiros .txt

## Bibliotecas a usar

Como estou a seguir a ligação acima referida, Keras e Tensorflow.

## Tratamento de dados

In [1]:
from os import listdir
from os.path import isfile, join
import sys
import collections, functools, operator
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback, ModelCheckpoint, ReduceLROnPlateau

Contar os caracteres presentes nas letras:

In [2]:
dataset_path = "./dataset"
files_in_dataset = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f))]
char_indices_list = []
indices_char_list = []

total_lyrics = ""
for filename in files_in_dataset:
    filepath = dataset_path +"/"+filename
    with open(filepath, 'r',encoding='utf8') as file:

        text = file.read().lower()
      
        total_lyrics += "\n\n"+text 

chars = sorted(list(set(total_lyrics))) # getting all unique chars

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print(char_indices)

{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, '+': 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, ';': 22, '?': 23, 'a': 24, 'b': 25, 'c': 26, 'd': 27, 'e': 28, 'f': 29, 'g': 30, 'h': 31, 'i': 32, 'j': 33, 'k': 34, 'l': 35, 'm': 36, 'n': 37, 'o': 38, 'p': 39, 'q': 40, 'r': 41, 's': 42, 't': 43, 'u': 44, 'v': 45, 'w': 46, 'x': 47, 'y': 48, 'z': 49, '{': 50, '}': 51, 'ª': 52, '«': 53, '»': 54, 'à': 55, 'á': 56, 'â': 57, 'ã': 58, 'ç': 59, 'é': 60, 'ê': 61, 'í': 62, 'ñ': 63, 'ò': 64, 'ó': 65, 'ô': 66, 'õ': 67, 'ú': 68, 'ü': 69, '\u2005': 70, '‘': 71, '’': 72, '“': 73, '”': 74, '…': 75, '\u205f': 76}


In [3]:
max_len = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(total_lyrics) - max_len, step):
    sentences.append(total_lyrics[i: i + max_len])
    next_chars.append(total_lyrics[i + max_len])

x = np.zeros((len(sentences), max_len, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

## Recurrent Neural Network

In [4]:
model = Sequential()
model.add(LSTM(256, input_shape=(max_len, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [5]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - max_len - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + max_len]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, max_len, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [6]:
filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [7]:
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)

callbacks = [print_callback, checkpoint, reduce_lr]

## Treinar a rede

In [ ]:
model.fit(x, y, batch_size=128, epochs=5, callbacks=callbacks, verbose=0)

In [ ]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - max_len - 1)
    generated = ''
    sentence = text[start_index: start_index + max_len]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, max_len, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [ ]:
import datetime
number_of_files = 5
numberchar = 250
date = datetime.datetime.now().date()

for counter in range(0, number_of_files):
    songFileName = "results/output_"+str(date)+"_"+str(numberchar)+"_"+str(counter)+".txt"
    with open(songFileName, 'w') as f:
        f.write(generate_text(numberchar, 0.2))

#print(generate_text(500, 0.2))